In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
#df = pd.read_csv(r"./data/cardekho_imputated.csv", index_col=[0])
df=pd.read_csv('cardekho_imputated.csv',index_col=[0])

In [3]:
df.drop('car_name', axis=1, inplace=True)
df.drop('brand', axis=1, inplace=True)

In [4]:
num_features= [features for features in df.columns if df[features].dtype !='O']
print('num_features:', len(num_features))
cat_features=[features for features in df.columns if df[features].dtype== 'O']
print('cat_features:',len(cat_features))
discrete_features=[feature for feature in num_features if len(df[feature].unique())<=25]
print('discrete_features:',len(discrete_features))
contious_features=[feature for feature in num_features if len(df[feature].unique())>25]
print('contious_features:',len(contious_features))

num_features: 7
cat_features: 4
discrete_features: 2
contious_features: 5


In [5]:
x = df.drop(['selling_price'], axis=1)
y = df['selling_price']

In [6]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
x['model']=le.fit_transform(df['model'])

In [7]:
num_features = x.select_dtypes(exclude="object").columns
one_hot_column=['seller_type','fuel_type','transmission_type']
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
std=StandardScaler()
one_hot=OneHotEncoder(drop='first')
preprocessor=ColumnTransformer([('Onehotencoder',one_hot,one_hot_column),('standard_scaler',std,num_features)],remainder='passthrough')

In [8]:
x=preprocessor.fit_transform(x)
x=pd.DataFrame(x)
x

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.519714,0.983562,1.247335,-0.000276,-1.324259,-1.263352,-0.403022
1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.225693,-0.343933,-0.690016,-0.192071,-0.554718,-0.432571,-0.403022
2,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.536377,1.647309,0.084924,-0.647583,-0.554718,-0.479113,-0.403022
3,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-1.519714,0.983562,-0.360667,0.292211,-0.936610,-0.779312,-0.403022
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,-0.666211,-0.012060,-0.496281,0.735736,0.022918,-0.046502,-0.403022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15406,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.508844,0.983562,-0.869744,0.026096,-0.767733,-0.757204,-0.403022
15407,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.556082,-1.339555,-0.728763,-0.527711,-0.216964,-0.220803,2.073444
15408,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.407551,-0.012060,0.220539,0.344954,0.022918,0.068225,-0.403022
15409,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.426247,-0.343933,72.541850,-0.887326,1.329794,0.917158,2.073444


In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)
x_train.shape, x_test.shape

((12328, 14), (3083, 14))

In [10]:
#MODEL SELECTION AND TRAINING
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [11]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [12]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "adaboost classifier": AdaBoostClassifier(),
    "adaboost regressor": AdaBoostRegressor()
   
}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(x_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    
    #print('='*35)
    print('\n')
    

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 553855.6665
- Mean Absolute Error: 268101.6071
- R2 Score: 0.6218
Model performance for Test set
- Root Mean Squared Error: 502543.5930
- Mean Absolute Error: 279618.5794
- R2 Score: 0.6645


Lasso
Model performance for Training set
- Root Mean Squared Error: 553855.6710
- Mean Absolute Error: 268099.2226
- R2 Score: 0.6218
Model performance for Test set
- Root Mean Squared Error: 502542.6696
- Mean Absolute Error: 279614.7461
- R2 Score: 0.6645


Ridge
Model performance for Training set
- Root Mean Squared Error: 553856.3160
- Mean Absolute Error: 268059.8015
- R2 Score: 0.6218
Model performance for Test set
- Root Mean Squared Error: 502533.8230
- Mean Absolute Error: 279557.2169
- R2 Score: 0.6645


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 325880.8558
- Mean Absolute Error: 91392.3102
- R2 Score: 0.8691
Model performance for Test set
- Root Mean Squared Error: 2

/Users/cybesama/Desktop/tryout/venv/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


adaboost classifier
Model performance for Training set
- Root Mean Squared Error: 787559.0221
- Mean Absolute Error: 350244.4030
- R2 Score: 0.2352
Model performance for Test set
- Root Mean Squared Error: 864673.1176
- Mean Absolute Error: 377968.5777
- R2 Score: 0.0068


adaboost regressor
Model performance for Training set
- Root Mean Squared Error: 453572.1620
- Mean Absolute Error: 342793.9429
- R2 Score: 0.7463
Model performance for Test set
- Root Mean Squared Error: 479835.3544
- Mean Absolute Error: 358004.6697
- R2 Score: 0.6941




In [17]:
#HYPERPARAMETER TUNING
adaboost_classifier_param={
    "n_estimators":[50,60,70,80,90],
    "algorithm":['SAMME','SAMME.R']
}
adaboost_regrressor_param={"n_estimators":[50,60,70,80],
                           "loss":['linear',"square","exponential"]}





In [26]:
randomcv_models = [ ("AdaBoost_classifier", AdaBoostClassifier(), adaboost_classifier_param),
                   ("AdaBoostRegressor",AdaBoostRegressor(),adaboost_regrressor_param)
                    ]

from sklearn.model_selection import RandomizedSearchCV

model_param = {}
for name, model, params in randomcv_models:
    random = RandomizedSearchCV(estimator=model,
                                   param_distributions=params,
                                   n_iter=100,
                                   cv=3,
                                   verbose=2,
                                   n_jobs=-1)
    random.fit(x_train, y_train)
    model_param[name] = random.best_params_

for model_name in model_param:
    print(f"---------------- Best Params for {model_name} -------------------")
    print(model_param[model_name])

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/Users/cybesama/Desktop/tryout/venv/lib/python3.9/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 10 is smaller than n_iter=100. Running 10 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/cybesama/Desktop/tryout/venv/lib/python3.9/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


[CV] END ...................algorithm=SAMME, n_estimators=50; total time=   4.3s
[CV] END ...................algorithm=SAMME, n_estimators=50; total time=   4.3s
[CV] END ...................algorithm=SAMME, n_estimators=50; total time=   4.5s
[CV] END ...................algorithm=SAMME, n_estimators=60; total time=   5.0s
[CV] END ...................algorithm=SAMME, n_estimators=60; total time=   5.2s
[CV] END ...................algorithm=SAMME, n_estimators=60; total time=   5.4s
[CV] END ...................algorithm=SAMME, n_estimators=70; total time=   6.1s
[CV] END ...................algorithm=SAMME, n_estimators=70; total time=   6.5s


/Users/cybesama/Desktop/tryout/venv/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


[CV] END ...................algorithm=SAMME, n_estimators=70; total time=   6.6s


...

[CV] END .................algorithm=SAMME.R, n_estimators=70; total time=  23.8s


/Users/cybesama/Desktop/tryout/venv/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


[CV] END .................algorithm=SAMME.R, n_estimators=70; total time=  24.6s
[CV] END .................algorithm=SAMME.R, n_estimators=70; total time=  20.7s
[CV] END .................algorithm=SAMME.R, n_estimators=80; total time=  22.5s
[CV] END .................algorithm=SAMME.R, n_estimators=80; total time=  21.8s
[CV] END .................algorithm=SAMME.R, n_estimators=80; total time=  20.6s
[CV] END .................algorithm=SAMME.R, n_estimators=90; total time=  22.3s
[CV] END .................algorithm=SAMME.R, n_estimators=90; total time=  22.1s
[CV] END .................algorithm=SAMME.R, n_estimators=90; total time=  18.8s


/Users/cybesama/Desktop/tryout/venv/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Users/cybesama/Desktop/tryout/venv/lib/python3.9/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 12 is smaller than n_iter=100. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END .......................loss=linear, n_estimators=60; total time=   0.4s
[CV] END .......................loss=linear, n_estimators=60; total time=   0.4s
[CV] END .......................loss=linear, n_estimators=50; total time=   0.5s
[CV] END .......................loss=linear, n_estimators=50; total time=   0.5s
[CV] END .......................loss=linear, n_estimators=50; total time=   0.5s
[CV] END .......................loss=linear, n_estimators=60; total time=   0.6s
[CV] END .......................loss=linear, n_estimators=70; total time=   0.5s
[CV] END .......................loss=linear, n_estimators=70; total time=   0.4s
[CV] END .......................loss=linear, n_estimators=70; total time=   0.7s
[CV] END .......................loss=linear, n_estimators=80; total time=   0.3s
[CV] END .......................loss=linear, n_estimators=80; total time=   0.4s
[CV] END .......................loss=square, n_e

In [28]:
models = {
    "AdaBoostclassifier":AdaBoostClassifier(n_estimators=50,algorithm="SAMME.R"),
    "Adaboost":AdaBoostRegressor(n_estimators=60,loss='linear')
}


In [29]:
for i in range(2):
    model = list(models.values())[i]
    model.fit(x_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    
    print(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    
    print('='*35)
    print('\n')

/Users/cybesama/Desktop/tryout/venv/lib/python3.9/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostclassifier
Model performance for Training set
- Root Mean Squared Error: 787559.0221
- Mean Absolute Error: 350244.4030
- R2 Score: 0.2352
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 864673.1176
- Mean Absolute Error: 377968.5777
- R2 Score: 0.0068


Adaboost
Model performance for Training set
- Root Mean Squared Error: 484949.1129
- Mean Absolute Error: 383125.4616
- R2 Score: 0.7100
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 517172.1476
- Mean Absolute Error: 399399.1442
- R2 Score: 0.6447


